# Problem Set 2
## Fundamentals of Simulation Methods

#### Author:    Elias Olofsson

#### Version information:
        2020-11-12: v.1.0. First public release. 



## Exercise 1 - Pitfalls of pseudo-random number generators

In [14]:
import numpy as np
import matplotlib.pyplot as plt

def randu(x):
    return (65539*x) % 2**31


n = 1000
seed = 11111

I2 = randu(seed)

for i in range(n):
    I1 = randu(I2)
    I2 = randu(I1)

    x = I1 / 2**31
    y = I2 / 2**31

    





214147157
1184882943
887005949
1100540151
767671013
1143615663
2100138509
160806439
1390944885
455960415
954676765
1624416855
1154410245
896644367
1432624941
525950343
999495829
1263421887
732552509
1614452663
